In [17]:
import pandas as pd
import random
import time
import numpy as np


from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold



from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")
A = pd.read_pickle("~/bge-analysis-simv3/A.pkl")


pa_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/eff-16-list1-red-pa.pkl")

##### compile the well numbers for each species for its presence 

species_ones = []
for i in range(16):
    species_ones.append(pa_redv1[pa_redv1[i]==1.0]["well number"].values)

ones_len = [] 
for ii in range(len(species_ones)):
    ones_len.append(len(species_ones[ii])) 

sst_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-16sp-FUNCTIONS/16-list1-allmetabols.pkl")
sst_redv1.head(4)

,2,3,5,6,7,9,10,11,15,16,...,18,19,20,21,22,well index,butyrate,acetate,lactate,succinate
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.977078,0.0,1.0,19.891379,3.802050,27.542614,2.614571
128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.520201,0.0,...,0.0,0.0,0.000000,0.000000,0.0,8.0,0.383275,2.611108,26.853592,2.116042
130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.284509,0.0,...,0.0,0.0,0.000000,0.640232,0.0,9.0,14.815900,4.867583,25.578500,3.605217
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.995363,0.000000,0.0,16.0,19.907547,5.276729,45.619753,2.863171


In [18]:
cv_splitsl = [3,5,7] 
max_depthl = [2,5,10,15,20]
ntrees = [100]
#max_featuresl = ["auto","sqrt","log2"]
max_featuresl = [0.33,0.5]

In [19]:
metabolite_list = ["butyrate","lactate","acetate","succinate"]

In [20]:
sst_redv1["acetate"]

2        3.802050
128      2.611108
130      4.867583
4        5.276729
6        6.547083
          ...    
1440    26.969250
289     18.775933
417     18.373800
2465    17.128000
368     12.576900
Name: acetate, Length: 187, dtype: float64

In [60]:
cv_splitsl = [3,5,7,10] 
max_depthl = [2,5,10,15,20]
ntrees = [100]
#max_featuresl = ["auto","sqrt","log2"]
max_featuresl = [0.33]

In [76]:
startt = time.time()
rfper = [] 
for iteras in range(5):
    trdf = pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-16sp-FUNCTIONS-v3/traindataRELfried"+str(iteras)+".pkl")
    tedf = pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-16sp-FUNCTIONS-v3/testdataRELfried"+str(iteras)+".pkl")
    for metabols in metabolite_list : 
        spdf = trdf[trdf["metabols"]==metabols]
        tesp = tedf[tedf["metabols"]==metabols]
        for m in [3,5,7,10] : 
            spidx = spdf[spdf["k-fold"]==m]
            spidxte = tesp[tesp["k-fold"]==m]
            for folds in range(m):
                trfold = spidx["train data"].iloc[folds]
                trfold.sort()
                trY =  sst_redv1.T[trfold].T[metabols]
                trX = pa_redv1[pa_redv1["well number"].isin(trfold)].sort_values(by="well number").T[:16].T
                tefold = spidxte["test data"].iloc[folds]
                tefold.sort()
                teY =  sst_redv1.T[tefold].T[metabols]
                teX = pa_redv1[pa_redv1["well number"].isin(tefold)].sort_values(by="well number").T[:16].T
                for md in max_depthl:
                    for n_est in ntrees:
                        for mfeatures in max_featuresl:
                                regr = RandomForestRegressor(max_depth=md,n_estimators=n_est,max_features=mfeatures)
                                regr.fit(trX,trY)
                                ypred = regr.predict(teX)
                                rfper.append([metabols,iteras,m,folds,md,n_est,mfeatures,ypred,teY.values,trfold,tefold])
                

        

In [77]:
rdf = pd.DataFrame(rfper,columns=["metabols","runs","k-fold","fold","md","trees","feats","ypred","yact","trfolds","tefolds"])

In [78]:
#rdf.to_pickle("RF-OV-functions.pkl")

In [80]:
import pandas as pd

rdf = pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-16sp-FUNCTIONS-v3/RF-OV-functions.pkl")

In [82]:
rdf[rdf["metabols"]=="butyrate"]

,metabols,runs,k-fold,fold,md,trees,feats,ypred,yact,trfolds,tefolds
0,butyrate,0,3,0,2,100,0.33,"[15.73741739052395, 16.521952640462604, 14.745...","[16.934522727272732, 22.711900000000004, 16.24...","[2, 3, 4, 5, 6, 8, 9, 12, 15, 16, 17, 18, 31, ...","[1, 7, 10, 20, 29, 36, 40, 45, 48, 66, 106, 12..."
1,butyrate,0,3,0,5,100,0.33,"[14.774128767138555, 17.237929029766963, 10.73...","[16.934522727272732, 22.711900000000004, 16.24...","[2, 3, 4, 5, 6, 8, 9, 12, 15, 16, 17, 18, 31, ...","[1, 7, 10, 20, 29, 36, 40, 45, 48, 66, 106, 12..."
2,butyrate,0,3,0,10,100,0.33,"[14.994151918836241, 17.60594632238483, 11.512...","[16.934522727272732, 22.711900000000004, 16.24...","[2, 3, 4, 5, 6, 8, 9, 12, 15, 16, 17, 18, 31, ...","[1, 7, 10, 20, 29, 36, 40, 45, 48, 66, 106, 12..."
3,butyrate,0,3,0,15,100,0.33,"[14.813492200000002, 18.211082648176482, 13.53...","[16.934522727272732, 22.711900000000004, 16.24...","[2, 3, 4, 5, 6, 8, 9, 12, 15, 16, 17, 18, 31, ...","[1, 7, 10, 20, 29, 36, 40, 45, 48, 66, 106, 12..."
4,butyrate,0,3,0,20,100,0.33,"[15.53293294999999, 17.96615893792158, 13.8566...","[16.934522727272732, 22.711900000000004, 16.24...","[2, 3, 4, 5, 6, 8, 9, 12, 15, 16, 17, 18, 31, ...","[1, 7, 10, 20, 29, 36, 40, 45, 48, 66, 106, 12..."
...,...,...,...,...,...,...,...,...,...,...,...
2120,butyrate,4,10,9,2,100,0.33,"[16.411550377144085, 25.663288549095522, 14.82...","[23.8199125, 48.019075, 0.23988333333333334, 2...","[1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 15, 16, 17, 1...","[5, 33, 64, 66, 167, 173, 211, 384, 418, 549, ..."
2121,butyrate,4,10,9,5,100,0.33,"[16.31900462465515, 34.697650636151735, 11.508...","[23.8199125, 48.019075, 0.23988333333333334, 2...","[1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 15, 16, 17, 1...","[5, 33, 64, 66, 167, 173, 211, 384, 418, 549, ..."
2122,butyrate,4,10,9,10,100,0.33,"[19.298890373982232, 40.28550601145684, 7.8478...","[23.8199125, 48.019075, 0.23988333333333334, 2...","[1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 15, 16, 17, 1...","[5, 33, 64, 66, 167, 173, 211, 384, 418, 549, ..."
2123,butyrate,4,10,9,15,100,0.33,"[20.692935479852054, 40.8730859955, 6.11473869...","[23.8199125, 48.019075, 0.23988333333333334, 2...","[1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 15, 16, 17, 1...","[5, 33, 64, 66, 167, 173, 211, 384, 418, 549, ..."
